# Advanced analysis with `pandas` and `FloPy`

This exercise, we will load and existing model into Flopy, run the model and then use [pandas](https://pandas.pydata.org/), [matplotlib](https://matplotlib.org/) and [numpy](https://www.numpy.org/) to look at the results and compare them to observed data. Including:
* reading tabular data from a file or url using the powerful `read_csv` method
* geting `pandas.DataFrame`s of Hydmod, SFR, and global mass balance output
* converting model times to real date-times to allow plotting against other temporally-referenced data
* quickly subsetting data by category, attribute values, times, index position, etc.
* computing quantiles and other basic statistics
* making plots using `matplotlib` and the built-in hooks to it in `pandas`

The example model is a TMR inset of the Little Plover River model set in central WI, which is documented by Bradbury and others (2017). The parent model files and report are avaible [here](https://wgnhs.wisc.edu/little-plover-river-groundwater-model/). The TMR inset encompasses the Little Plover River down to the Hoover Ave. bridge in Plover, WI, and the calendar year of 2014. Unconfined groundwater flow in the underlying glacial outwash, and base flow in the Little Plover River are simulated. Constant heads are used for the perimeter boundaries. The properties are mostly the same as the parent model.

![](../data/lpr_inset/figs/lpr_inset.png)


**Reference:**
Bradbury, K.R., Fienen, M.N., Kniffin, M.L., Krause, J.J., Westenbroek, S.M., Leaf, A.T., and Barlow, P.M., 2017, Groundwater flow model for the Little Plover River basin in Wisconsin’s Central Sands: Wisconsin Geological and Natural History Survey Bulletin 111, 82 p.

In [ ]:
import flopy
fm = flopy.modflow
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
plt.rcParams['figure.figsize'] = (11, 8.5)

### inputs

In [ ]:
namefile = 'lpr_inset.nam'
model_ws = '../data/lpr_inset/'
listingfile = '{}/lpr_inset.list'.format(model_ws)
gagefile = '{}/5400625lpr.ggo'.format(model_ws)
start_datetime = '2014-01-01'

gage_data_url = 'https://nwis.waterdata.usgs.gov/wi/nwis/uv?&format=rdb&site_no=05400625&period=&begin_date=2014-01-01&end_date=2014-12-31'

### load the model

In [ ]:
m = fm.Modflow.load(namefile, model_ws=model_ws)
m.get_package_list()

In [ ]:
m

### plot the K field in layer 1

In [ ]:
plt.imshow(m.upw.hk.array[0], vmax=350)
plt.colorbar()

### plot recharge for the model period
convert from feet per day to annual inches

In [ ]:
m.rch.rech.array.shape

In [ ]:
r_in = m.rch.rech.array[:, 0, :, :] * 12
plt.imshow(r_in.sum(axis=0) * 30.4)
plt.colorbar()

### spatial mean of recharge through time
convert from feet per day to average monthly totals (in inches) for model area

In [ ]:
plt.bar(range(12), r_in.mean(axis=(1, 2)) * 30.4)

### set the executable

on Windows: `../bin/mfnwt.exe`

In [ ]:
m.exe_name = 'mfnwt'

In [ ]:
m.run_model()

## Looking at results using pandas

#### loading the gage file using `read_csv`

In [ ]:
ggo = pd.read_csv(gagefile, skiprows=1, header=None, delim_whitespace=True, names=['time', 'stage', 'flow'])

In [ ]:
ggo.head()

#### convert times in gage output to real date times

In [ ]:
start_ts = pd.Timestamp(start_datetime)
start_ts

In [ ]:
pd.to_timedelta(ggo.time, unit='D').head()

In [ ]:
ggo['datetime'] = pd.to_timedelta(ggo.time, unit='D') + start_ts
ggo.index = ggo.datetime
ggo.head()

In [ ]:
ggo.flow.plot()

### What if we only want to plot flow results for the last timestep of each stress period?

#### make lists of the period and timestep at each row in the gage package output

In [ ]:
m.dis.nstp.array

In [ ]:
kstp = []
kper = []
for i, nstp in enumerate(m.dis.nstp.array):
    for j in range(nstp):
        kstp.append(j)
        kper.append(i)
len(kstp)

In [ ]:
ggo['kstp'] = kstp
ggo['kper'] = kper
ggo.head()

### Get the last timesteps using `groupby`

In [ ]:
ggo_last = ggo.groupby(ggo.kper).last()
ggo_last.index = ggo_last.datetime
ggo_last

In [ ]:
ax = ggo_last.flow.plot(marker='o')
ggo.flow.plot(ax=ax)

Some Flopy utilities can return model results in DataFrames.

### Get model mass balance information from the listing file

In [ ]:
from flopy.utils import MfListBudget

In [ ]:
mfl = MfListBudget('{}/{}.list'.format(model_ws, m.name))

In [ ]:
df_flux, df_vol = mfl.get_dataframes(start_datetime='01-01-2014')

In [ ]:
df_flux.head()

### plot some of the components against gage output

If we want to have more control over a plot, it often makes sense to use `matplotlib.pyplot` instead of the `DataFrame.plot()` methods.

In [ ]:
nbars = 2
width = 1/(nbars+2)
xvalues = np.arange(12)+1

fig, axes = plt.subplots(2, 1)
ax = axes.flat[0]
ax.bar(xvalues-width, -df_flux.WELLS_OUT, width=width, label='pumping', color='C1')
ax.bar(xvalues, df_flux.RECHARGE_IN, width=width, label='recharge', color='C0', alpha=0.5)
ax.bar(xvalues+width, df_flux.CONSTANT_HEAD_IN, width=width, label='constant head in', color='C2')
ax.bar(xvalues+width, -df_flux.CONSTANT_HEAD_OUT, width=width, label='constant head out', color='DarkGreen')

ax2 = axes.flat[1]
ax2.plot(xvalues, ggo_last.flow/86400)
ax.legend(bbox_to_anchor=(1., 1.05))

### Compare gage output to actual gage data from the web

In [ ]:
gage_data_url

### get data for site 5400625 in rdb format, using `read_csv`

In [ ]:
df = pd.read_csv(gage_data_url, skiprows=30, header=None, sep='\t')
df.head()

In [ ]:
'../data/lpr_inset/05400625.txt'

### convert times to pandas datetimes

flow values are in column 4 in cubic feet per second

In [ ]:
df.index = pd.to_datetime(df[2])
df.head()

### convert monthly gage package results to cfs

In [ ]:
ggo_last['flow_cfs'] = ggo_last['flow'] / 86400
ggo_last.head()

In [ ]:
ax = ggo_last.flow_cfs.plot()
df[4].plot()

### convert gage results to monthly averages for flows less than q threshold

align timestamps with gage results

In [ ]:
qthresh = 0.5
quant = df[4].quantile(qthresh)
quant

In [ ]:
df[4].loc[df[4] < quant].plot()

In [ ]:
dfq = df[4].loc[df[4] < quant]
dfm = dfq.groupby(dfq.index.month).mean()
dfm

### create end of month times for monthly gage data averages so they plot with gage package output

In [ ]:
start_ts

In [ ]:
pd.date_range('2014-03', '2014-12', freq='M')

In [ ]:
dfm.index = pd.date_range('2014-03', '2015-01', freq='M')
dfm

In [ ]:
ax = ggo_last.flow_cfs.plot(label='gage output')
dfm.plot(ax=ax, label='data for 05400625')
ax.legend()

In [ ]:
dfm

In [ ]:
ggo_last

## Hydmod

In [ ]:
from flopy.utils import HydmodObs

In [ ]:
hydobj = HydmodObs('{}/{}.hyd.bin'.format(m.model_ws, m.name))
hyd = pd.DataFrame(hydobj.get_data())
hyd.head()

## Look at SFR water balance output (time pending)

In [ ]:
from flopy.utils import SfrFile

In [ ]:
sfrobj = SfrFile('{}/{}.sfr.out'.format(m.model_ws, m.name))

### get dataframe of SFR results

In [ ]:
sfrdata = sfrobj.df.copy()

In [ ]:
sfrdata.head()

### plot the groundwater/surfacewater interactions for a segment

get the distance information from the SFR package input

In [ ]:
seg11 = sfrdata.loc[(sfrdata.segment == 11) & (sfrdata.kstpkper == (4, 6)), ['Qout', 'Qaquifer']]

# get the reach data for SFR package
rd = pd.DataFrame(m.sfr.reach_data)
rd.head()

In [ ]:
dist = rd.loc[rd.iseg == 11, 'rchlen'].cumsum().tolist()
dist[0:10]

In [ ]:
seg11.index = dist

In [ ]:
ax = seg11.plot()
ax.axhline(0, color='k', zorder=-1, lw=0.5)

### make a quick and dirty map of groundwater/surface water interactions

In [ ]:
arr = np.zeros((m.nrow, m.ncol), dtype=float) * np.nan

i = sfrdata.loc[sfrdata.kstpkper == (4, 6), 'i']
j = sfrdata.loc[sfrdata.kstpkper == (4, 6), 'j']
qgw = sfrdata.loc[sfrdata.kstpkper == (4, 6), 'Qaquifer']
arr[i, j] = qgw

In [ ]:
plt.imshow(arr)
plt.colorbar()

In [ ]:
losing = arr.copy()
losing[losing <= 0] = np.nan
gaining = arr.copy()
gaining[gaining >= 0] = np.nan
im = plt.imshow(losing, cmap='Reds')
im2 = plt.imshow(gaining, cmap='Blues_r')
plt.colorbar(im)
plt.colorbar(im2)